# Module 6 Challenge
### Deliverable 2: Create a Customer Travel Destinations Map

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,East London,ZA,-33.0153,27.9116,72.54,64,0,17.27,clear sky
1,1,Olafsvik,IS,64.8945,-23.7142,26.56,68,82,15.70,snow
2,2,Mataura,NZ,-46.1927,168.8643,50.49,98,100,2.04,overcast clouds
3,3,Tasiilaq,GL,65.6145,-37.6368,6.96,74,4,3.76,clear sky
4,4,Kaitangata,NZ,-46.2817,169.8464,48.78,94,96,5.91,overcast clouds


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 80


In [10]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
#clean_df
filtered_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
filtered_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,East London,ZA,-33.0153,27.9116,72.54,64,0,17.27,clear sky
5,5,Busselton,AU,-33.6500,115.3333,76.08,56,32,6.17,scattered clouds
11,11,Kupang,ID,-10.1667,123.5833,77.58,100,40,2.30,scattered clouds
13,13,Atuona,PF,-9.8000,-139.0333,78.80,72,9,19.39,clear sky
14,14,Port Elizabeth,ZA,-33.9180,25.5701,72.10,73,20,25.32,few clouds
15,15,San Patricio,US,28.0170,-97.5169,77.09,70,7,14.90,clear sky
19,19,Mbandaka,CD,0.0487,18.2603,75.20,89,66,6.69,light rain
25,25,Ponta Do Sol,PT,32.6667,-17.1000,71.01,85,59,5.68,broken clouds
26,26,Rikitea,PF,-23.1203,-134.9692,76.66,79,69,13.58,broken clouds
28,28,Avarua,CK,-21.2078,-159.7750,75.25,73,100,5.75,overcast clouds


In [11]:
# 4a. Determine if there are any empty rows.
filtered_df.count()

City_ID                153
City                   153
Country                152
Lat                    153
Lng                    153
Max Temp               153
Humidity               153
Cloudiness             153
Wind Speed             153
Current Description    153
dtype: int64

In [12]:
filtered_df.isnull().sum()

City_ID                0
City                   0
Country                1
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [13]:
filtered_df.notnull().sum()

City_ID                153
City                   153
Country                152
Lat                    153
Lng                    153
Max Temp               153
Humidity               153
Cloudiness             153
Wind Speed             153
Current Description    153
dtype: int64

In [14]:
filtered_df.loc[(clean_df["Country"].isnull())]

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
84,84,Walvis Bay,NaN,-22.9575,14.5053,76.15,50,12,19.57,few clouds


In [15]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
# Only 1 record was found with a NULL country code
clean_df = filtered_df.dropna(how='any')

In [16]:
# check if there is any NULL value again
clean_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [19]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,East London,ZA,72.54,clear sky,-33.0153,27.9116,
5,Busselton,AU,76.08,scattered clouds,-33.6500,115.3333,
11,Kupang,ID,77.58,scattered clouds,-10.1667,123.5833,
13,Atuona,PF,78.80,clear sky,-9.8000,-139.0333,
14,Port Elizabeth,ZA,72.10,few clouds,-33.9180,25.5701,
15,San Patricio,US,77.09,clear sky,28.0170,-97.5169,
19,Mbandaka,CD,75.20,light rain,0.0487,18.2603,
25,Ponta Do Sol,PT,71.01,broken clouds,32.6667,-17.1000,
26,Rikitea,PF,76.66,broken clouds,-23.1203,-134.9692,
28,Avarua,CK,75.25,overcast clouds,-21.2078,-159.7750,


In [20]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]        
    except (IndexError):
        print("Hotel not found... Skipping.")

Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.


In [33]:
# List all with no Hotel Name specified
hotel_df.loc[(hotel_df["Hotel Name"] == '')]

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
113,Aripuana,BR,76.48,light rain,-9.1667,-60.6333,
274,Gamba,GA,77.20,light rain,-2.6500,10.0000,
429,Pueblo Bello,CO,79.30,overcast clouds,8.2041,-76.5260,
459,Bongandanga,CD,76.44,light rain,1.5000,21.0500,
486,Kasongo-Lunda,CD,77.83,overcast clouds,-6.4667,16.8167,
523,Sembakung,ID,78.73,overcast clouds,3.8167,117.2500,
576,Tiznit,MA,73.40,clear sky,29.5833,-9.5000,
635,Mundo Nuevo,MX,75.72,clear sky,18.7000,-95.5000,


In [36]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[(hotel_df["Hotel Name"] != '')]
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,East London,ZA,72.54,clear sky,-33.0153,27.9116,Tu Casa
5,Busselton,AU,76.08,scattered clouds,-33.6500,115.3333,Observatory Guest House
11,Kupang,ID,77.58,scattered clouds,-10.1667,123.5833,Hotel On The Rock by Prasanthi
13,Atuona,PF,78.80,clear sky,-9.8000,-139.0333,Villa Enata
14,Port Elizabeth,ZA,72.10,few clouds,-33.9180,25.5701,39 On Nile Guest House
15,San Patricio,US,77.09,clear sky,28.0170,-97.5169,"Motel 6 Sinton, TX"
19,Mbandaka,CD,75.20,light rain,0.0487,18.2603,Guest House Epervier
25,Ponta Do Sol,PT,71.01,broken clouds,32.6667,-17.1000,Hotel do Campo
26,Rikitea,PF,76.66,broken clouds,-23.1203,-134.9692,People ThankYou
28,Avarua,CK,75.25,overcast clouds,-21.2078,-159.7750,Paradise Inn


In [37]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [39]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [41]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

# 11b. Display the figure
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))